# Import Module

In [1]:
import numpy as np 
import pandas as pd

# Deeplearning
import torch
import torchvision
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Data load
import os
import random
from torch.utils.data import TensorDataset, Dataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Check Data

In [3]:
# 이미지 확인하기
image=cv2.imread('../input/pizza-not-pizza/pizza_not_pizza/not_pizza/1005746.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
Image.fromarray(image) 

cv2 모듈을 사용해 이미지 파일을 불러오면 채널이 BGR으로 등록된다. 이를 cvtColor를 사용해 RGB로 바꿔준다.

cv2.imread('path')를 사용하면 path(경로)에 있는 데이터를 행렬(array)형태로 불러온다.

In [4]:
# 데이터 개수 확인
not_pizza = os.listdir('../input/pizza-not-pizza/pizza_not_pizza/not_pizza')

pizza = os.listdir('../input/pizza-not-pizza/pizza_not_pizza/pizza')

print('not_pizza list 길이 : ',len(not_pizza));print('pizza list 길이 : ',len(pizza))

print('\n')

print(not_pizza[0]) # 이미지 파일 이름이 리스트에 저장된 것을 확인

os.listdir(path)는 path 경로에 있는 값들을 리스트화하는 함수이다. 

# Load Data

In [5]:
image=cv2.imread('../input/pizza-not-pizza/pizza_not_pizza/not_pizza/1005746.jpg')
image.shape

PyTorch의 ImageTensor는 **(채널 x 높이 x 너비)** 의 구조를 갖고 있다. 
하지만 위의 코드를 보면 알다시피 python에서 3차원 tensor를 표현할 때는 **(높이 x 너비 x 채널)** 의 구조를 갖는다. 이를 해결하기 위해 **torchvision .transforms**를 사용한다. 
해당 모듈을 사용하면 tensor 구조 뿐만 아니라 밝기 정도 또한  **[0 ~ 255] 에서 [0 ~ 1] 으로 스케일링**된다.

In [6]:
#주워니
data_transform = {
    'train' :  transforms.Compose([
        transforms.Resize(300),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(300),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

--------
Resize 옵션을 사용해 이미지의 높이와 너비를 300 으로 만들고

RandomCrop 옵션을 사용해 이미지의 높이와 너비를 224로 자른다.

--------

In [7]:
path = "/kaggle/input/pizza-not-pizza/pizza_not_pizza/"

images_dataset = {x : datasets.ImageFolder(root = path, transform = data_transform[x]) for x in ['train', 'val']}

train_dataloader = DataLoader(images_dataset['train'], batch_size= 3, shuffle=True, num_workers=4)
val_dataloader = DataLoader(images_dataset['val'], batch_size= 3, shuffle=True, num_workers=4)

# Model 

In [8]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # L1 ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 32)
        #    Pool     -> (?, 112, 112, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 ImgIn shape=(?, 112, 112, 32)
        #    Conv      ->(?, 112, 112, 64)
        #    Pool      ->(?, 56, 56, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        
        # L3 ImgIn shape=(?, 56, 56, 64)
        #    Conv      ->(?, 56, 56, 128)
        #    Pool      ->(?, 28, 28, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(28 * 28 * 128, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [9]:
model = CNN().to(device)

In [10]:
learning_rate = 0.001
epochs = 100
batch_size = 10

In [11]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# ckeck loss

In [12]:

'''
loss_arr = []
for i in range(epochs):
    for j, [image,label] in enumerate(train_dataloader):
        x = image.to(device)
        y = label.to(device)
        
        optimizer.zero_grad()
        
        output = model.forward(x)
        
        loss = criterion(output, y)
        loss.backward()
        
        optimizer.step()
        
        if j % 10 == 0 :
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())        
'''

# ckeck accuracy

In [13]:
y_true = list()
y_pred = list()

with torch.no_grad():
    for j, [image,label] in enumerate(val_dataloader):
        x = image.to(device)
        y = label.to(device)
        
        pred = model(x).argmax(dim=-1)
        
        for i in range(len(pred)):
            y_true.append(y[i].item())
            y_pred.append(pred[i].item())
            
np.mean(y_pred)

np.mean(y_true)

In [15]:
loss_arr = []
for i in range(epochs):
    for j, [image,label] in enumerate(train_dataloader):
        x = image.to(device)
        y = label.to(device)
        
        model.eval()
'''        
        with torch.no_grad():
            for j, [image,label] in enumerate(val_dataloader):
                x = image.to(device)
                y = label.to(device)

                pred = model(x).argmax(dim=-1)

                for i in range(len(pred)):
                    y_true.append(y[i].item())
                    y_pred.append(pred[i].item())
'''
#model.eval(x,y)

In [ ]:
y_true = list()
y_pred = list()

with torch.no_grad():
    for j, [image,label] in enumerate(val_dataloader):
        x = image.to(device)
        y = label.to(device)
        
        pred = model(x).argmax(dim=-1)
        
        for i in range(len(pred)):
            y_true.append(y[i].item())
            y_pred.append(pred[i].item())
            
np.mean(y_pred)

np.mean(y_true)